In [2]:
import torch
from torchvision.datasets import MNIST
from torch.utils.data import DataLoader
from torchvision import transforms

In [3]:
data_transform = transforms.Compose([transforms.ToTensor(), 
                                     transforms.Normalize((0.5,), (0.5,))])

train_data = MNIST(root="./data", train=True, download=True, transform=data_transform)
test_data = MNIST(root="./data", train=False, download=True, transform=data_transform)

In [4]:
print("Training data length: ", len(train_data))
print("Test data length: ", len(test_data))

Training data length:  60000
Test data length:  10000


In [5]:
batch_size = 64

train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=True)

In [6]:
from model import Classifier

net = Classifier()
print(net)
#net.load_model("./mnist_model.pt")

Classifier(
  (conv1): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (dropout): Dropout(p=0.25, inplace=False)
  (fc1): Linear(in_features=800, out_features=400, bias=True)
  (fc2): Linear(in_features=400, out_features=10, bias=True)
)


In [6]:
def train(num_epochs, model):
    for epoch in range(num_epochs):
        running_loss = 0.0
        for i, data in enumerate(train_loader):
            #unpack
            inputs, labels = data

            #reset the gradients
            optimizer.zero_grad()

            #forward
            outputs = model(inputs)

            #loss
            loss = criterion(outputs, labels)

            #calculate backprop weights
            loss.backward()

            #apply learning
            optimizer.step()

            #add loss
            running_loss += loss.item()

            if (i % 100) == 99:
                mean_loss = running_loss / 100
                print("Epoch ", epoch + 1, "; Batch", i + 1, "; Train loss: ", mean_loss)
                running_loss = 0.0

In [9]:
import torch.optim as optim
import torch.nn as nn

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.005, momentum=0.9)

In [10]:
epochs = 30

train(epochs, net)
print("Finished training!")

Epoch  1 ; Batch 100 ; Train loss:  0.03492684114200529
Epoch  1 ; Batch 200 ; Train loss:  0.03570411839755252
Epoch  1 ; Batch 300 ; Train loss:  0.038326805524993686
Epoch  1 ; Batch 400 ; Train loss:  0.0429396550077945
Epoch  1 ; Batch 500 ; Train loss:  0.031962334311101584
Epoch  1 ; Batch 600 ; Train loss:  0.037949443323304875
Epoch  1 ; Batch 700 ; Train loss:  0.03952873055357486
Epoch  1 ; Batch 800 ; Train loss:  0.04050223772530444
Epoch  1 ; Batch 900 ; Train loss:  0.044642827765783294
Epoch  2 ; Batch 100 ; Train loss:  0.03288874818943441
Epoch  2 ; Batch 200 ; Train loss:  0.04430500017013401
Epoch  2 ; Batch 300 ; Train loss:  0.03714826587587595
Epoch  2 ; Batch 400 ; Train loss:  0.040992767363786695
Epoch  2 ; Batch 500 ; Train loss:  0.031777181772049516
Epoch  2 ; Batch 600 ; Train loss:  0.037035333197563886
Epoch  2 ; Batch 700 ; Train loss:  0.0360169585084077
Epoch  2 ; Batch 800 ; Train loss:  0.0337024169054348
Epoch  2 ; Batch 900 ; Train loss:  0.041531

In [11]:
torch.save(net.state_dict(), "./mnist_model.pt")

In [21]:
## testing the model by hand

net.eval()
image, label = next(iter(test_loader))

import numpy as np

num = np.random.randint(batch_size)
x = torch.unsqueeze(image[num], 0)
print(x.shape)
with torch.no_grad():
    print("Predicted:", torch.argmax(net(x)).item(), ", True:", label[num].item())

torch.Size([1, 1, 28, 28])
Predicted: 2 , True: 2


In [22]:
def test_model(test_dataloader, model):
    correct_pred = 0
    model.eval()

    for images, labels in test_dataloader:
        for i in range(len(labels)):
            image = torch.unsqueeze(images[i], 0)

            with torch.no_grad():
                output = model(image)

            label_predicted = torch.argmax(output).item()
            label_true = labels[i].item()

            if label_predicted == label_true:
                correct_pred += 1

    total = len(test_data)

    return correct_pred / total


In [23]:
accuracy = test_model(test_loader, net)

print(f"The accuracy of the model on the test dataset: {accuracy * 100}%")

The accuracy of the model on the test dataset: 99.39%
